# FFTW C2C Python MPI 576

- https://numpy.org/doc/stable/reference/routines.fft.html
- https://mpi4py-fft.readthedocs.io/en/latest/parallel.html

In [41]:
#-----------------------------------------------------------------------

- module load  anaconda3/2020.11
- module load  mathlibs/fftw/3.3.8_openmpi-3.1_gnu
- conda activate /scratch/app/anaconda3/2020.11
- conda activate $pwd/env2

In [1]:
! module list

Currently Loaded Modulefiles:
  1) anaconda3/2020.11
  2) openmpi/gnu/3.1.4
  3) mathlibs/fftw/3.3.8_openmpi-3.1_gnu


In [4]:
%%writefile pc2cp.py
import numpy as np, time as tm
from mpi4py_fft import PFFT, newDistArray
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

t0 = tm.time()    # time measurement

# data
L = M = N = 576
NA = np.array([N, M, L], dtype=int)
f = PFFT(comm, NA, dtype=np.complex128, backend='pyfftw')
u = newDistArray(f, False)
for k in range (u.shape[2]) :
    for j in range(u.shape[1]) :
        for i in range(u.shape[0]) :
            u[i, j, k] = i + j + k + 3
u.real = np.sin ( u.real )
u.imag = 0

t1 = tm.time()    # time measurement

# FFT
uf = f.forward(u, normalize=False)

# checksum
S  = np.array(0, dtype=np.complex128)
Sn = np.array(np.sum(uf), dtype=np.complex128)
comm.Reduce([Sn, MPI.DOUBLE_COMPLEX], [S, MPI.DOUBLE_COMPLEX],
            op=MPI.SUM, root=0)

t2 = tm.time()    # time measurement

if rank == 0 :
    print(f"S: {S*1E-5:.0f}", end='')
    print(f" | L: {L:0g}", end='')
    print(f" | N: {size:0g}", end='')
    print(f" | TS: {t1-t0:.4f}", end='')
    print(f" | TP: {t2-t1:.4f}", end='')
    print(f" | TT: {t2-t0:.4f}")

Overwriting pc2cp.py


## Check

In [5]:
! time mpiexec -n 4 python pc2cp.py

S: 270-0j | L: 576 | N: 4 | TS: 32.9811 | TP: 6.1715 | TT: 39.1526

real	0m40.213s
user	2m20.558s
sys	0m18.978s


In [6]:
! time mpiexec -n 16 python pc2cp.py

S: 270-0j | L: 576 | N: 16 | TS: 8.5875 | TP: 1.4231 | TT: 10.0105

real	0m13.470s
user	2m20.098s
sys	0m23.762s


## Copy to /scratch

In [7]:
%%bash
dst=/scratch${PWD#"/prj"}
cp pc2cp.py $dst

## Batch script

In [8]:
%%writefile pc2cp.srm
#!/bin/bash
#SBATCH --job-name pc2cp       # Job name
#SBATCH --partition cpu_small  # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:10:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
cd $SLURM_SUBMIT_DIR
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment
echo '-- modules ----------------------------'
cd
dir=/scratch${PWD#"/prj"}
cd $dir
source $dir/env2/etc/profile.d/conda.sh
conda activate $dir/env2
conda activate --stack $dir/env3
cd $dir/fft
                                              
# Executable
EXEC="python pc2cp.py"

# Start
echo '-- run --------------------------------'
echo '$ srun --mpi=pmi2 -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun --mpi=pmi2 -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting pc2cp.srm


## Check

In [9]:
! sbatch --partition=cpu_dev --ntasks=96 pc2cp.srm

Submitted batch job 1332923


In [10]:
! squeue -n pc2cp -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1332923    cpu_dev  PD  0:00     4   96


In [13]:
! squeue -n pc2cp -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS


In [14]:
! cat /scratch${PWD#"/prj"}/slurm-1332923.out

- Job ID: 1332923
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1247 sdumont1248 sdumont1249 sdumont1250
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 96 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 96 | TS: 15.4266 | TP: 6.2516 | TT: 21.6782
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


## Run

### 1 of (1, 4, 16, 24, 48, 72, 96)

In [15]:
! sbatch --ntasks=1 pc2cp.srm
! sbatch --ntasks=1 pc2cp.srm
! sbatch --ntasks=1 pc2cp.srm

Submitted batch job 1332941
Submitted batch job 1332942
Submitted batch job 1332943


In [8]:
! cat /scratch${PWD#"/prj"}/slurm-1332941.out
! cat /scratch${PWD#"/prj"}/slurm-1332942.out
! cat /scratch${PWD#"/prj"}/slurm-1332943.out

- Job ID: 1332941
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1276
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 1 | TS: 159.8739 | TP: 19.2747 | TT: 179.1486
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332942
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 1 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 1 | TS: 170.5870 | TP: 19.3180 | TT: 189.9049
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332943
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: fft
- 

### 4 of (1, 4, 16, 24, 48, 72, 96)

In [16]:
! sbatch --ntasks=4 pc2cp.srm
! sbatch --ntasks=4 pc2cp.srm
! sbatch --ntasks=4 pc2cp.srm

Submitted batch job 1332944
Submitted batch job 1332945
Submitted batch job 1332946


In [7]:
! cat /scratch${PWD#"/prj"}/slurm-1332944.out
! cat /scratch${PWD#"/prj"}/slurm-1332945.out
! cat /scratch${PWD#"/prj"}/slurm-1332946.out

- Job ID: 1332944
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 4 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 4 | TS: 37.8654 | TP: 3.9770 | TT: 41.8424
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332945
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1276
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 4 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 4 | TS: 40.2809 | TP: 8.7450 | TT: 49.0258
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332946
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 4
- Dir from which sbatch was invoked: fft
- List o

### 16 of (1, 4, 16, 24, 48, 72, 96)

In [17]:
! sbatch --ntasks=16 pc2cp.srm
! sbatch --ntasks=16 pc2cp.srm
! sbatch --ntasks=16 pc2cp.srm

Submitted batch job 1332947
Submitted batch job 1332948
Submitted batch job 1332949


In [6]:
! cat /scratch${PWD#"/prj"}/slurm-1332947.out
! cat /scratch${PWD#"/prj"}/slurm-1332948.out
! cat /scratch${PWD#"/prj"}/slurm-1332949.out

- Job ID: 1332947
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 16 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 16 | TS: 9.5749 | TP: 1.0548 | TT: 10.6297
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332948
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 16 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 16 | TS: 10.8813 | TP: 1.3761 | TT: 12.2574
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332949
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 16
- Dir from which sbatch was invoked: fft
- 

### 24 of (1, 4, 16, 24, 48, 72, 96)

In [18]:
! sbatch --ntasks=24 pc2cp.srm
! sbatch --ntasks=24 pc2cp.srm
! sbatch --ntasks=24 pc2cp.srm

Submitted batch job 1332950
Submitted batch job 1332951
Submitted batch job 1332952


In [5]:
! cat /scratch${PWD#"/prj"}/slurm-1332950.out
! cat /scratch${PWD#"/prj"}/slurm-1332951.out
! cat /scratch${PWD#"/prj"}/slurm-1332952.out

- Job ID: 1332950
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 24 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 24 | TS: 6.1370 | TP: 1.1616 | TT: 7.2987
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332951
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1276
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 24 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 24 | TS: 6.7884 | TP: 0.8618 | TT: 7.6502
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332952
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 24
- Dir from which sbatch was invoked: fft
- Lis

### 48 of (1, 4, 16, 24, 48, 72, 96)

In [19]:
! sbatch --ntasks=48 pc2cp.srm
! sbatch --ntasks=48 pc2cp.srm
! sbatch --ntasks=48 pc2cp.srm

Submitted batch job 1332953
Submitted batch job 1332954
Submitted batch job 1332955


In [4]:
! cat /scratch${PWD#"/prj"}/slurm-1332953.out
! cat /scratch${PWD#"/prj"}/slurm-1332954.out
! cat /scratch${PWD#"/prj"}/slurm-1332955.out

- Job ID: 1332953
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275 sdumont1276
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 48 python pc2cp.py
-- output -----------------------------
S: 270+0j | L: 576 | N: 48 | TS: 5.4049 | TP: 8.1324 | TT: 13.5373
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332954
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1278 sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 48 python pc2cp.py
-- output -----------------------------
S: 270+0j | L: 576 | N: 48 | TS: 5.3402 | TP: 7.1480 | TT: 12.4882
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332955
- Tasks per node:
- # of nodes in the job: 2
- # of tasks: 48
- Dir from which sba

### 72 of (1, 4, 16, 24, 48, 72, 96)

In [20]:
! sbatch --ntasks=72 pc2cp.srm
! sbatch --ntasks=72 pc2cp.srm
! sbatch --ntasks=72 pc2cp.srm

Submitted batch job 1332956
Submitted batch job 1332957
Submitted batch job 1332958


In [3]:
! cat /scratch${PWD#"/prj"}/slurm-1332956.out
! cat /scratch${PWD#"/prj"}/slurm-1332957.out
! cat /scratch${PWD#"/prj"}/slurm-1332958.out

- Job ID: 1332956
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 72
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1278 sdumont1286 sdumont1491
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 72 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 72 | TS: 1.8776 | TP: 9.3624 | TT: 11.2400
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332957
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 72
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275 sdumont1276 sdumont1278
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 72 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 72 | TS: 2.1981 | TP: 7.0923 | TT: 9.2903
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332958
- Tasks per node:
- # of nodes in the job: 3
- # of tasks: 

### 96 of (1, 4, 16, 24, 48, 72, 96)

In [21]:
! sbatch --ntasks=96 pc2cp.srm
! sbatch --ntasks=96 pc2cp.srm
! sbatch --ntasks=96 pc2cp.srm

Submitted batch job 1332959
Submitted batch job 1332960
Submitted batch job 1332961


In [2]:
! cat /scratch${PWD#"/prj"}/slurm-1332959.out
! cat /scratch${PWD#"/prj"}/slurm-1332960.out
! cat /scratch${PWD#"/prj"}/slurm-1332961.out

- Job ID: 1332959
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275 sdumont1276 sdumont1278 sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 96 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 96 | TS: 1.5857 | TP: 7.3273 | TT: 8.9130
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332960
- Tasks per node:
- # of nodes in the job: 4
- # of tasks: 96
- Dir from which sbatch was invoked: fft
- List of nodes allocated to the job: sdumont1275 sdumont1276 sdumont1278 sdumont1286
-- modules ----------------------------
-- run --------------------------------
$ srun --mpi=pmi2 -n 96 python pc2cp.py
-- output -----------------------------
S: 270-0j | L: 576 | N: 96 | TS: 1.6761 | TP: 7.2206 | TT: 8.8967
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- Job ID: 1332961
- Tasks per node:
- # of nodes in th

In [22]:
! squeue --partition=cpu_small -h -t pending,running -r | wc -l

275


In [23]:
! squeue -u $(whoami) -h -t pending,running -r | wc -l

21


In [24]:
! squeue -n pc2cp -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
           1332941  cpu_small  PD  0:00     1    1
           1332942  cpu_small  PD  0:00     1    1
           1332943  cpu_small  PD  0:00     1    1
           1332944  cpu_small  PD  0:00     1    4
           1332945  cpu_small  PD  0:00     1    4
           1332946  cpu_small  PD  0:00     1    4
           1332947  cpu_small  PD  0:00     1   16
           1332948  cpu_small  PD  0:00     1   16
           1332949  cpu_small  PD  0:00     1   16
           1332950  cpu_small  PD  0:00     1   24
           1332951  cpu_small  PD  0:00     1   24
           1332952  cpu_small  PD  0:00     1   24
           1332953  cpu_small  PD  0:00     2   48
           1332954  cpu_small  PD  0:00     2   48
           1332955  cpu_small  PD  0:00     2   48
           1332956  cpu_small  PD  0:00     3   72
           1332957  cpu_small  PD  0:00     3   72
           1332958  cpu_small  PD  0:00     3   72
           1332959  cpu_small  

In [1]:
! squeue -n pc2cp -o "%.18i  %.9P  %.2t %.5M %.5D %.4C"

             JOBID  PARTITION  ST  TIME NODES CPUS
